In [2]:
# Explore turnstile data to look at how people exit stations before a large event
# and enter stations afterwards. Goal is to assist police in redirecting traffic flow 
# through high density stations as a means to improve pedestrian safety
# Consider Grand Central Station on two events: 4th Of Jult Fireworks (to the east of the statiobn) and 
# St. Patrick's Day Parade (to the west of the station). How do people exit the station to get to the event
# how do people re-enter the station after the event?
# what time do people start to crowd the station? 
# how does this compare to a typical weekday?

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [35]:
# various options in pandas
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 75)
pd.set_option('display.precision', 3)

In [36]:
mta_df = pd.read_csv('../01_Data/turnstile_180707.txt')   #read CSV of data including July 4th 2018

In [37]:
mta_df.rename(index=str, columns={"C/A": "CA"}, inplace = True)  #rename C/A because the slash causes errors

In [30]:
mta_df.head(10)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2018,00:00:00,REGULAR,6675523,2262828
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2018,04:00:00,REGULAR,6675538,2262836
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2018,08:00:00,REGULAR,6675554,2262863
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2018,12:00:00,REGULAR,6675652,2262941
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2018,16:00:00,REGULAR,6675826,2262980
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2018,20:00:00,REGULAR,6676110,2263019
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/01/2018,00:00:00,REGULAR,6676241,2263043
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/01/2018,04:00:00,REGULAR,6676248,2263046
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/01/2018,08:00:00,REGULAR,6676256,2263058
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/01/2018,12:00:00,REGULAR,6676339,2263098


In [48]:
#I took photos of 33rd st to see what the data means. Checking values here

station_df = mta_df[mta_df.STATION == '33 ST']
print(station_df["CA"].unique())
ca_df = station_df[station_df.CA == 'R231']
print(ca_df["UNIT"].unique())
unit_df = ca_df[ca_df.UNIT == 'R176']
print(unit_df["SCP"].unique())

# it looks like the SCP is the individual turnstiles



['R231' 'R231A' 'R232' 'R232A']
['R176']
['00-00-00' '00-00-01' '00-00-02' '00-00-03' '00-00-04' '00-00-05']


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
# Pull GCT data
gct_df = mta_df[mta_df.STATION == 'GRD CNTRL-42 ST'] #Isolate only Grand Central Station data

In [11]:
gct_74_df = gct_df[gct_df.DATE == '07/04/2018']  #isolate only July 4th

In [20]:
gct_74_df["DATETIME"] = pd.to_datetime(gct_df['DATE'] + ' ' + gct_df['TIME'] )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
gct_74_df.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
151016,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,01:00:00,REGULAR,1474370,4172636,2018-07-04 01:00:00
151017,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,05:00:00,REGULAR,1474372,4172645,2018-07-04 05:00:00
151018,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,09:00:00,REGULAR,1474385,4172766,2018-07-04 09:00:00
151019,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,13:00:00,REGULAR,1474430,4173064,2018-07-04 13:00:00
151020,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,17:00:00,REGULAR,1474541,4173376,2018-07-04 17:00:00


In [15]:
# Since each turnstile has 6 timed data points per day, dividing by 6 allows you to figure out 
# how many individual turnstiles there are in each station

gct_74_df["CA"].value_counts()/6

R238     18.000
R236     11.833
R240     11.000
R238A     8.000
R237      7.000
R241A     5.000
R237B     4.000
Name: CA, dtype: float64

In [56]:
gct_74_df[gct_74_df.CA == 'R236']

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
151016,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,01:00:00,REGULAR,1474370,4172636,2018-07-04 01:00:00
151017,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,05:00:00,REGULAR,1474372,4172645,2018-07-04 05:00:00
151018,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,09:00:00,REGULAR,1474385,4172766,2018-07-04 09:00:00
151019,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,13:00:00,REGULAR,1474430,4173064,2018-07-04 13:00:00
151020,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,17:00:00,REGULAR,1474541,4173376,2018-07-04 17:00:00
151021,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,21:00:00,REGULAR,1474625,4173909,2018-07-04 21:00:00
151058,R236,R045,00-00-01,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,01:00:00,REGULAR,328902,705497,2018-07-04 01:00:00
151059,R236,R045,00-00-01,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,05:00:00,REGULAR,328902,705501,2018-07-04 05:00:00
151060,R236,R045,00-00-01,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,09:00:00,REGULAR,328916,705608,2018-07-04 09:00:00
151061,R236,R045,00-00-01,GRD CNTRL-42 ST,4567S,IRT,07/04/2018,13:00:00,REGULAR,328955,705878,2018-07-04 13:00:00


In [58]:
gct_74_df.groupby('SCP').count()

,CA,UNIT,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
SCP,,,,,,,,,,,
00-00-00,24,24,24,24,24,24,24,24,24,24,24
00-00-01,24,24,24,24,24,24,24,24,24,24,24
00-00-02,18,18,18,18,18,18,18,18,18,18,18
00-00-03,18,18,18,18,18,18,18,18,18,18,18
00-00-04,18,18,18,18,18,18,18,18,18,18,18
00-00-05,12,12,12,12,12,12,12,12,12,12,12
00-00-06,6,6,6,6,6,6,6,6,6,6,6
00-00-07,6,6,6,6,6,6,6,6,6,6,6
00-03-00,18,18,18,18,18,18,18,18,18,18,18


In [52]:
list = []
list.append(gct_74_df['CA'] == 'R236')
list

[151016    True
 151017    True
 151018    True
 151019    True
 151020    True
 151021    True
 151058    True
 151059    True
 151060    True
 151061    True
 151062    True
 151063    True
 151100    True
 151101    True
 151102    True
 151103    True
 151104    True
 151105    True
 151142    True
 151143    True
 151144    True
 151145    True
 151146    True
 151147    True
 151184    True
 151185    True
 151186    True
 151187    True
 151188    True
 151189    True
 151226    True
 151227    True
 151228    True
 151229    True
 151230    True
 151231    True
 151268    True
           ... 
 153456    True
 153493    True
 153494    True
 153495    True
 153496    True
 153497    True
 153498    True
 153535    True
 153536    True
 153537    True
 153538    True
 153539    True
 153540    True
 153577    True
 153578    True
 153579    True
 153580    True
 153581    True
 153582    True
 153619    True
 153620    True
 153621    True
 153622    True
 153623    True
 153624 

In [16]:
gct_74_df["UNIT"].value_counts()/6

R046    33.000
R047    15.000
R045    11.833
R048     5.000
Name: UNIT, dtype: float64

In [54]:
gct_74_df["SCP"].value_counts()

00-00-01    24
00-00-00    24
00-00-03    18
00-03-03    18
00-03-01    18
00-00-04    18
00-03-00    18
00-00-02    18
00-03-02    17
01-00-01    12
00-06-01    12
01-00-02    12
01-00-03    12
01-00-00    12
00-06-00    12
00-00-05    12
00-03-04    12
02-00-04     6
02-03-00     6
00-05-01     6
02-00-00     6
01-00-04     6
00-03-05     6
00-03-08     6
02-00-02     6
00-03-07     6
00-03-06     6
02-03-02     6
00-06-02     6
00-00-07     6
00-05-00     6
02-03-01     6
02-00-01     6
01-00-05     6
01-00-06     6
00-00-06     6
02-00-03     6
Name: SCP, dtype: int64

In [46]:
gct_74_df[gct_74_df.UNIT == 'R045']['SCP'].unique()

array(['00-00-00', '00-00-01', '00-00-02', '00-00-03', '00-00-04',
       '00-00-05', '00-03-00', '00-03-01', '00-03-02', '00-03-03',
       '00-06-00', '00-06-01'], dtype=object)

In [22]:
gct_74_df["TIME"].unique()

array(['01:00:00', '05:00:00', '09:00:00', '13:00:00', '17:00:00',
       '21:00:00', '00:00:00', '04:00:00', '08:00:00', '12:00:00',
       '16:00:00', '20:00:00'], dtype=object)